### Create Binary Label

In [8]:
from pyspark.sql import functions as F

import numpy as np

import pyspark
from pyspark import pandas as ps
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark import SparkContext, SparkConf
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.sql.functions import col, substring
from pyspark.sql.types import StructType, StructField, \
StringType, IntegerType, TimestampType, DateType, FloatType
from pyspark.sql.window import Window

In [4]:
conf = pyspark.SparkConf().setAll([\
            ('spark.app.name', 'Glucose_Analysis_Spark')])\
            .set('spark.sql.shuffle.partitions', '1500')
spark = SparkSession.builder.config(conf=conf)\
    .getOrCreate()  

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/03 02:35:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/03 02:35:15 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/05/03 02:35:15 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [20]:
df = spark.read.parquet('/cephfs/stepped_glucose_data/step0_load_daily/parquet_0/part-00000-1c9c2511-4c2e-40c2-b7d1-7827039567e8-c000.snappy.parquet')

In [21]:
df.show(5)

+--------------------+-----+--------------------+---------------------+------------------+
|           PatientId|Value|  GlucoseDisplayTime|GlucoseDisplayTimeRaw|GlucoseDisplayDate|
+--------------------+-----+--------------------+---------------------+------------------+
|1Jxgxke6R3Uh2c9aR...|  0.0|2022-02-01 14:45:...| 2022-02-01T14:45:...|        2022-02-01|
|toBStbTTYI2GU28Yd...|  0.0|2022-02-01 17:46:...| 2022-02-01T17:46:...|        2022-02-01|
|+XAhHhm+BkhqusxsZ...|  0.0|2022-02-01 14:58:...| 2022-02-01T14:58:...|        2022-02-01|
|+XAhHhm+BkhqusxsZ...|  0.0|2022-01-31 22:53:...| 2022-01-31T22:53:...|        2022-01-31|
|+XAhHhm+BkhqusxsZ...|  0.0|2022-01-31 22:38:...| 2022-01-31T22:38:...|        2022-01-31|
+--------------------+-----+--------------------+---------------------+------------------+
only showing top 5 rows



In [81]:
test = df.filter(df.PatientId == '8NQv3W7U0vS/STkzxLUcmTJwXkbZUN3M/Rs/B6FuS/k=')\
        .select('PatientId', "Value", F.percent_rank().over(Window.partitionBy(df['PatientId'])\
                                                .orderBy(df['Value'])).alias("percent_rank"))

test.filter((test.percent_rank > .09) & (test.percent_rank < .10)).orderBy(test['percent_rank']).show(truncate=False)

+--------------------------------------------+-----+-------------------+
|PatientId                                   |Value|percent_rank       |
+--------------------------------------------+-----+-------------------+
|8NQv3W7U0vS/STkzxLUcmTJwXkbZUN3M/Rs/B6FuS/k=|72.0 |0.09278350515463918|
|8NQv3W7U0vS/STkzxLUcmTJwXkbZUN3M/Rs/B6FuS/k=|72.0 |0.09278350515463918|
|8NQv3W7U0vS/STkzxLUcmTJwXkbZUN3M/Rs/B6FuS/k=|72.0 |0.09278350515463918|
|8NQv3W7U0vS/STkzxLUcmTJwXkbZUN3M/Rs/B6FuS/k=|72.0 |0.09278350515463918|
|8NQv3W7U0vS/STkzxLUcmTJwXkbZUN3M/Rs/B6FuS/k=|72.0 |0.09278350515463918|
+--------------------------------------------+-----+-------------------+



In [83]:
default_lower = 70
default_upper = 180
lower_10, upper_90  = df.approxQuantile('Value', [.1, .9], 0)

In [87]:
lower_10

92.0

In [88]:
upper_90

248.0

In [84]:
df=df.withColumn('y_Binary', F.when(((F.col('Value') > default_upper) & (F.col('Value') > upper_90)), 1)\
            .when(((F.col('Value') < default_lower) & (F.col('Value') < lower_10)), 1)\
                .otherwise(0))

In [92]:
df.filter((df.Value != 0) & (df.y_Binary == 1)).show(30)

+--------------------+-----+-------------------+---------------------+------------------+--------+
|           PatientId|Value| GlucoseDisplayTime|GlucoseDisplayTimeRaw|GlucoseDisplayDate|y_Binary|
+--------------------+-----+-------------------+---------------------+------------------+--------+
|f/odpYRZXAvqV2R8A...| 64.0|2022-02-01 03:39:12| 2022-02-01T03:39:...|        2022-02-01|       1|
|Fj39UPMcINcICT0cb...|282.0|2022-02-01 00:28:21| 2022-02-01T00:28:...|        2022-02-01|       1|
|Fj39UPMcINcICT0cb...|276.0|2022-02-01 00:48:21| 2022-02-01T00:48:...|        2022-02-01|       1|
|Fj39UPMcINcICT0cb...|283.0|2022-01-31 23:58:21| 2022-01-31T23:58:...|        2022-01-31|       1|
|Fj39UPMcINcICT0cb...|285.0|2022-02-01 00:18:21| 2022-02-01T00:18:...|        2022-02-01|       1|
|Fj39UPMcINcICT0cb...|270.0|2022-01-31 23:28:21| 2022-01-31T23:28:...|        2022-01-31|       1|
|Fj39UPMcINcICT0cb...|283.0|2022-01-31 23:53:21| 2022-01-31T23:53:...|        2022-01-31|       1|
|Fj39UPMcI